# Running Auctions On-Chain

### Auction Contract
Below is our implementation of a first price auction, which helps facilitate auctioning off an NFT. The auctioneer creates an auction by making an instance of the firstPriceAuction contract, inputting the auction duration, and information about the NFT being auctioned off. During the auction duration, bidders can call the bid() function, which stores their bids. The pay() function is callable after the auction has ended, and conducts the transaction between the auctioneer and the winner.

```
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;
import "@openzeppelin/contracts/token/ERC721/IERC721.sol";

contract firstPriceAuction {
    address payable public auctionOwner;
    uint public auctionEndTime;
    uint public highestBid; // should this and below be changed to something not public? idk
    address payable public highestBidder;
    bool public hasPaid; // default value is false
    IERC721 public nft;
    uint public nftId;

    //mapping(address => uint) public bids;
    mapping(address => bool) public hasBid; //might not need this, gas prices incentivize not spamming too hard

    constructor(uint biddingTime, address thenft, uint thenftid) {
        // need to verify that msg.sender owns thenft
        // make sure nftid and nft check out
        require(IERC721(thenft).ownerOf(thenftid) == msg.sender, "Auctioneer does not own NFT");
        auctionOwner = payable(msg.sender);
        auctionEndTime = block.timestamp + biddingTime;
        nft = IERC721(thenft);
        nftId = thenftid;
    }

    function bid(uint thebid) public {
        require(block.timestamp <= auctionEndTime, "Auction has ended");
        require(thebid > 0, "Only Positive Bids Allowed");
        require(!hasBid[msg.sender], "Already Bid");
        hasBid[msg.sender] = true;
        if (thebid > highestBid) {
            highestBid = thebid;
            highestBidder = payable(msg.sender);
        }
    }

    function pay() public payable {
        require(block.timestamp >= auctionEndTime, "Auction has not ended yet");
        require(msg.sender == highestBidder, "You are not the highest bidder");
        require(msg.value == highestBid, "Insufficient Eth Transferred");
        require(!hasPaid, "Payment already made");
        hasPaid = true;
        auctionOwner.transfer(highestBid);
        nft.safeTransferFrom(auctionOwner, msg.sender, nftId);
    }
}
```

NOTE: this contact was never deployed to the test net for testing due to high gas fees (we tested using Remix VM)

### Instructions for Testing
Auctioneer (Class Instructor):

0. (Optional) Deploy the NFT contract in BasicNFT.sol, then call mintNFT(). The corresponding ID of the minted NFT starts at 0.
1.   Deploy the above first price auction contract with bidding time (in seconds), the NFT address, and the NFT ID. 
2. Ensure that the auction contract has permissions to transfer the NFT. If using BasicNFT from step 0, call setApprovalForAll passing in the address of the auction contract, and the boolean True.
3.   Once the auction is done and the winning bidder has called pay(), check MetaMask to see the funds update. Also verify that you are no longer the owner of the NFT 

Bidders (Students):

1. Once the auction is up and running, call the bid() function, passing in the amount in wei you want to bid on the NFT.
2. Once the auction is over, call the pay() function, passing in the amount you bid through msg.value. If you are the winning bidder, the transaction will go through (verify with MetaMask, and call ownerOf() on the NFT). 



### Other Auction Formats

Feel free to experiment with other types of auction formats (https://github.com/a-paulraj/On-Chain-Auctions), including the second price auction, descending auction, and a sealed bid auction (same as first price, but uses keccak256 hash function to seal bids, and has bidders call a reveal function to validate their bids once the auction is done).
